# Movie Recommender System

In this example you will create a movie recommender system.

The system will extract feature vectors from metadata about films using SentenceTransformers, import those vectors into Milvus, with the metadata. When a user submits information about movies they're interested in, you'll search Milvus for similar films and provide searchers movie info from Redis using the results.

## Requirements
- Python 3.x.
- Docker
- A system with at least 32GB of RAM, or a Zilliz cloud account

## Data

In this project, you'll use [The Movies Dataset from Kaggle](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data). This dataset contains metadata on more than 45k movies.

The dataset has several files, but you'll only need **movies_metadata.csv,** the main Movies Metadata file. You can use this notebook as a starting point and modify it to take advantage of the rest of this dataset.

# Requirements

First, install the Python packages needed for this project.

In [ ]:
! python -m pip install pymilvus redis pandas sentence_transformers kaggle

## Download dataset

Now you'll download the dataset. You'll use the [Kaggle API](https://github.com/Kaggle/kaggle-api) to retrieve the data. 

Set your login information below, or download **kaggle.json** to a location where the API will find it. 

In [ ]:
%env KAGGLE_USERNAME=username
%env KAGGLE_KEY=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

%env TOKENIZERS_PARALLELISM=true

Download the data and unzip it to the **dataset** directory.

In [ ]:
import kaggle

kaggle.api.authenticate()
kaggle.api.dataset_download_files('rounakbanik/the-movies-dataset', path='dataset', unzip=True)

## Milvus Server

You're going to create vector embeddings from the movie's descriptions. So, you need a way to store, index, and search on those embeddngs. That's where Milvus comes in.

This is a relatively large dataset, at least for a server running on a personal computer. So, you may want to use a Zilliz Cloud instance to store these vectors.

But, if you want to stay with a local instance, you can download a docker compose configuration and run that.

Here's how to get the compose file, and start the server.

In [ ]:
! wget https://github.com/milvus-io/milvus/releases/download/v2.3.0/milvus-standalone-docker-compose.yml -O docker-compose.yml
! docker-compose up -d

But, if you want to use cloud, sign up for an account [here](https://cloud.zilliz.com).

In [ ]:
import pandas as pd

movies=pd.read_csv('dataset/movies_metadata.csv',low_memory=False)
movies.shape

Youhave more than 45k movies, with 24 columns of metadata.

List the columns.

In [ ]:
movies.columns

There's no need to store all these columns in Milvus. Trim them down to the metatdata we want to store with the vectors and remove any items that are missing critical fields.

In [ ]:
from math import isnan
from pprint import pprint

trimmed_movies = movies[["title", "overview", "release_date", "genres"]]
trimmed_movies.head(4)


unclean_movies_dict = trimmed_movies.to_dict('records')
print('{} movies'.format(len(unclean_movies_dict)))
movies_dict = []

for movie in unclean_movies_dict:
    if  movie["overview"] == movie["overview"] and movie["release_date"] == movie["release_date"] and movie["genres"] == movie["genres"] and movie["title"] == movie["title"]:
        movies_dict.append(movie)

print('{} movies'.format(len(movies_dict)))


Now, it's time to connect to Milvus so you can start uploading data.

Here's the code for connecting to a cloud instance. Replace the URI and TOKEN with the correct values for your instance. 

You can find them in your Zilliz dashboard:
![image.png](cluster_info.png)

In [ ]:
from pymilvus import *

milvus_uri="XXXXXXX"
token="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
connections.connect("default",
                        uri=milvus_uri,
                        token=token)
print("Connected!")

So, with the meta data stored in Redis, it's time to calculate the embeddings and add them to Milvus.

First, you need a collection to store them in. Create a simple one that stores the title and embeddings for in the **Movies** field, while also allowing dynamic fields. You'll use the dynamic fields for metadata.

Then, you'll index the embedding field to make searches more efficent.

In [70]:
COLLECTION_NAME = 'film_vectors'
PARTITION_NAME = 'Movie'

# Here's our record schema
"""
"title": Film title,
"overview": description,
"release_date": film release date,
"genres": film generes,
"embedding": embedding
"""

id = FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=500, is_primary=True)
field = FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=384)

schema = CollectionSchema(fields=[id, field], description="movie recommender: film vectors", enable_dynamic_field=True)

if utility.has_collection(COLLECTION_NAME): # drop the same collection created before
    collection = Collection(COLLECTION_NAME)
    collection.drop()
    
collection = Collection(name=COLLECTION_NAME, schema=schema)
print("Collection created.")

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

print("Collection indexed!")
 

Collection created.


alloc_timestamp unimplemented, ignore it


Collection indexed!


Now, you need a function to create the embeddings.

The primary artifact for movie information is the overview, but including the genre and release date in complete sentences may help with search accuracy.

Create a transformer and call it from a simple function:
- extract the id field
- creates an embed from the overview, genre and release date
- inserts the vector into Milvus.

You'll reuse the **build_genres** function below for searching.

In [ ]:
from sentence_transformers import SentenceTransformer
import ast

def build_genres(data):
    genres = data['genres']
    genre_list = ""
    entries= ast.literal_eval(genres)
    genres = ""
    for entry in entries:
        genre_list = genre_list + entry["name"] + ", "
    genres += genre_list
    genres = "".join(genres.rsplit(",", 1))
    return genres

transformer = SentenceTransformer('all-MiniLM-L6-v2')

def embed_movie(data):
    embed = "{} Released on {}. Genres are {}.".format(data["overview"], data["release_date"], build_genres(data))    
    embeddings = transformer.encode(embed)
    return embeddings

Now, you can create the embeddings. This dataset is too large to send to Milvus in a single insert statement, but sending them one at a time would create unnecessary network traffic and add too much time. So, this code uses batches. You can play with the batch size to suit your individual needs and preferences.

In [ ]:
# Loop counter for batching and showing progress
j = 0
batch = []

for movie_dict in movies_dict:
    try:
        movie_dict["embedding"] = embed_movie(movie_dict)
        batch.append(movie_dict)
        j += 1
        if j % 5 == 0:
            print("Embedded {} records".format(j))
            collection.insert(batch)
            print("Batch insert completed")
            batch=[]
    except Exception as e:
        print("Error inserting record {}".format(e))
        pprint(batch)
        break

collection.insert(movie_dict)
print("Final batch completed")
print("Finished with {} embeddings".format(j))

Now you can search for movies that match viewer criteria. To do this, you need a few more functions.

First, you need a transformer to convert the user's search string to an embedding. For this, **embed_search** takes their criteria and passed it to the same transformer you used to populate Milvus.

By setting the title and overview fields in the return set, you can simply print the result set for the user.

Finally, **search_for_movies** performs the actual vector search, using the other two functions for support.

In [ ]:
collection.load() # load collection memory before search

# Set search parameters
topK = 5
SEARCH_PARAM = {
    "metric_type":"L2",
    "params":{"nprobe": 20},
}


def embed_search(search_string):
    search_embeddings = transformer.encode(search_string)
    return search_embeddings


def search_for_movies(search_string):
    user_vector = embed_search(search_string)
    return collection.search([user_vector],"embedding",param=SEARCH_PARAM, limit=topK, expr=None, output_fields=['title', 'overview'])
    

So, put this search to work!

This search is looking for 1990s comedies with Vampires. The first hit is exactly that, but as the vector distance increases you can see that the films move further away from what you're looking for.

You can play around with different search criteria.

In [ ]:
from pprint import pprint


search_string = "A comedy from the 1990s set in a hospital. The main characters are in their 20s and are trying to stop a vampire."
results = search_for_movies(search_string)

for hits in iter(results):
    for hit in hits:
        print(hit.entity.get('title'))
        print(hit.entity.get('overview'))
        print("-------------------------------")
